## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import entropy

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
        ### Compute KL divergence for hist and spec
        try: 
            kl_hist=entropy(dict_sample['hist_val'][:20],dict_val['hist_val'][:20])
        except: kl_hist=np.nan
        try: 
            kl_spec=entropy(dict_sample['spec_val'][:63],dict_val['spec_val'][:63])
        except: kl_spec=np.nan

        chisqr_dict.update({'kldiv_hist':kl_hist})
        chisqr_dict.update({'kldiv_spec':kl_spec})
    
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2','kldiv_hist','kldiv_spec']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
num_bkgnd=8000
### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[10000:10000+num_bkgnd][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


In [7]:
df_runs=pd.DataFrame([])

In [8]:
# ### Load raw images ( a different set of input images for comparison)
# dict1={}
# dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
# images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
# dict1['num_imgs']=images.shape[0]
# ### Compute spectrum and histograms
# dict_sample=f_compute_hist_spect(images,bins)
# ### Compute chi squares
# dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

# dict1.update(dict_sample)
# dict1.update(dict_chisqrs)
# del(images)
# df_runs=df_runs.append(dict1,ignore_index=True)


In [9]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [10]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201213_173446_lambda0.1_good_models/'
prefix='pytorch_1_'

lst=[(3, 11890), (4,13360), (10,33240), (10, 32650)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
#     fname=glob.glob(parent_dir+'images/'+'gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    fname=glob.glob(parent_dir+'images/'+'inference_0_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]

    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(1000, 128, 128)
0.9962296 -0.99917257
(1000, 128, 128)
0.9958533 -0.99951965
(1000, 128, 128)
0.9939648 -0.99748814
(1000, 128, 128)
0.9950605 -0.99199754


In [11]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201214_121039_lambda1.0_very_good_models/'
prefix='pytorch_2_'

lst=[(3, 10460), (10, 32880), (10, 33230), (15, 47070), (16, 52700), (17, 55860)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'inference_0_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


(1000, 128, 128)
0.9944289 -0.9999998
(1000, 128, 128)
0.99345326 -0.99883777
(1000, 128, 128)
0.9875575 -0.99967635
(1000, 128, 128)
0.98994416 -0.99848044
(1000, 128, 128)
0.9897598 -0.99705565
(1000, 128, 128)
0.9836897 -0.9975221


In [12]:
parent_dir+'images/'+'gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step)

'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201214_121039_lambda1.0_very_good_models/images/gen_img_epoch-17_step-55860.npy'

In [13]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201220_073758_lambda2.0_very_good_models/'
prefix='pytorch_3_'

lst=[(7, 22010), (10, 33550), (11, 35970), (16, 50810), (19, 60320), (25, 79050)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
#     fname=glob.glob(parent_dir+'images/'+'inference_0_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]

    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(64, 128, 128)
0.98392755 -0.999743
(64, 128, 128)
0.983372 -0.9996928
(64, 128, 128)
0.9834111 -0.9997707
(64, 128, 128)
0.9818723 -0.99815124
(64, 128, 128)
0.9761708 -0.9970999
(64, 128, 128)
0.98125273 -0.9981004


In [14]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201220_073758_lambda2.0_very_good_models/'
prefix='pytorch_4_'

lst=[(7, 22010), (10, 33550), (11, 35970), (16, 50810), (19, 60320), (25, 79050)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'inference_1_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


(1000, 128, 128)
0.99429137 -0.999994
(1000, 128, 128)
0.98931396 -0.99992794
(1000, 128, 128)
0.9872785 -0.9998696
(1000, 128, 128)
0.98622954 -0.99888253
(1000, 128, 128)
0.9895319 -0.9988311
(1000, 128, 128)
0.9900511 -0.99919343


In [15]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201220_073758_lambda2.0_very_good_models/'
prefix='pytorch_5_'

lst=[(28, 87959), (35, 112169)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'best*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(2000, 128, 128)
0.9895394 -0.9992044
(2000, 128, 128)
0.98758763 -0.9996125


In [16]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'kldiv_hist', 'kldiv_spec', 'label', 'num_imgs',
       'spec_sdev', 'spec_val'],
      dtype='object')

In [17]:

col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1','kldiv_hist','kldiv_spec','chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [18]:
df_runs

,label,chi_1,chi_1a,chi_spec1,kldiv_hist,kldiv_spec,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,keras_0,0.001751,0.000088,0.052795,0.000098,0.000033,23.822981,0.000523,0.001140,0.000027,4.107815,5.345283,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.003081765924486072, 0.0014312717459353363, ...","[1.1146491994998429, 0.43918919254001465, 0.04...","[42790.708515013386, 21535.157649232824, 14415...","[85906.63125591779, 60940.73459686716, 43971.7...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_1,0.001890,0.000097,0.066858,0.000055,0.000075,21.185616,0.000432,0.001362,0.000027,4.141878,5.868547,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0031456442853499706, 0.0014634180705460243,...","[1.1148820753244182, 0.4395964969226546, 0.044...","[44993.59584742486, 22703.883553150186, 16263....","[86439.30616832257, 60426.54691713405, 45079.8...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_2,0.002163,0.001067,0.020309,0.000331,0.000379,25.792137,0.000453,0.000644,0.000814,4.056808,1.242998,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0032299157726108983, 0.001488031590088233, ...","[1.1439634352627102, 0.42365239595815146, 0.04...","[41984.635273342275, 22377.328892028356, 14242...","[82519.15650125599, 59058.62442671653, 43854.1...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_3,0.002804,0.000827,0.047763,0.000288,0.000064,20.922330,0.000678,0.001299,0.000606,4.013538,5.103600,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0029680794378720322, 0.0013843083015912224,...","[1.0973298711470636, 0.4486810023923401, 0.045...","[45458.16732584579, 21884.70881964065, 14530.5...","[88968.60526771784, 62091.84879396162, 45370.3...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,pytorch_1_3-11890,0.016477,0.001656,0.464277,0.001233,0.000606,25.858514,0.005848,0.008973,0.000637,3.356407,23.658896,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0033754217532636333, 0.0016014703278977756,...","[1.1404960876694838, 0.42627387070672174, 0.04...","[46314.099039724744, 22620.433623518624, 15390...","[80488.54666225467, 56446.059667409994, 42557....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pytorch_1_4-13360,0.032353,0.005139,0.159234,0.001444,0.001068,28.081766,0.001300,0.025913,0.003918,3.166807,8.019884,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0020090705796929355, 0.0009721552524328763,...","[1.0642011822493573, 0.4672217666431568, 0.044...","[50901.141808125554, 23843.27834475123, 15290....","[78634.57799212456, 53762.150585867166, 39940....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pytorch_1_10-33240,0.013545,0.005362,0.856489,0.001595,0.001113,28.925948,0.002286,0.005897,0.003916,3.212316,75.436411,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0022728262925506906, 0.0010015221318092694,...","[1.063482247234686, 0.46551498906920286, 0.048...","[59927.04662677373, 31824.05077485303, 19990.5...","[93665.20471976128, 63530.008071346936, 46108....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pytorch_1_10-32650,0.032520,0.018360,0.324173,0.004968,0.001887,25.314499,0.002769,0.011392,0.014667,3.036371,28.476342,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0021636449429141176, 0.001067254258009924, ...","[1.0120878264783408, 0.49380646629449265, 0.04...","[46468.02183423206, 22975.63709879165, 14588.5...","[80144.22531076203, 54729.875300763466, 40151....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pytorch_2_3-10460,0.023237,0.017950,1.498381,0.004269,0.008250,35.149209,0.003855,0.001432,0.006807,4.156721,52.165261,"[-1.031746031584782, -0.3099415213241682, 0.30...","[0.0037009117345392314, 0.0016172020893072596,...","[1.1875398132437616, 0.3943260237888215, 0.055...","[87446.45670259123, 41395.57278159278, 24100.3...","[135590.76173461997, 87985.99237620502, 5

### Compare chi-squares

In [19]:
chi_keys=['label','chi_1','chi_spec1','kldiv_hist','kldiv_spec', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,kldiv_hist,kldiv_spec,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,keras_0,0.001751,0.052795,0.000098,0.000033,500.0,0.000088,0.000523,0.001140,0.000027,4.107815,5.345283,23.822981
1,keras_1,0.001890,0.066858,0.000055,0.000075,500.0,0.000097,0.000432,0.001362,0.000027,4.141878,5.868547,21.185616
2,keras_2,0.002163,0.020309,0.000331,0.000379,500.0,0.001067,0.000453,0.000644,0.000814,4.056808,1.242998,25.792137
3,keras_3,0.002804,0.047763,0.000288,0.000064,500.0,0.000827,0.000678,0.001299,0.000606,4.013538,5.103600,20.922330
4,pytorch_1_3-11890,0.016477,0.464277,0.001233,0.000606,1000.0,0.001656,0.005848,0.008973,0.000637,3.356407,23.658896,25.858514
5,pytorch_1_4-13360,0.032353,0.159234,0.001444,0.001068,1000.0,0.005139,0.001300,0.025913,0.003918,3.166807,8.019884,28.081766
6,pytorch_1_10-33240,0.013545,0.856489,0.001595,0.001113,1000.0,0.005362,0.002286,0.005897,0.003916,3.212316,75.436411,28.925948
7,pytorch_1_10-32650,0.032520,0.324173,0.004968,0.001887,1000.0,0.018360,0.002769,0.011392,0.014667,3.036371,28.476342,25.314499
8,pytorch_2_3-10460,0.023237,1.498381,0.004269,0.008250,1000.0,0.017950,0.003855,0.001432,0.006807,4.156721,52.165261,35.149209
9,pytorch_2_10-32880,0.001816,0.086587,0.000067,0.000409,1000.0,0.000166,0.001205,0.000445,0.000079,3.809940,2.865929,28.639090


## Plot

In [22]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
        
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=300)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')


#     plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.legend(loc='best')

    if plot_type in ['hist','spec']:     plt.yscale('log')    
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')




In [23]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('keras_0', 'keras_1', 'keras_2', 'ker…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [ ]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

In [ ]:
df_runs.iloc[0].fname